In [8]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
data = pd.read_csv('/home/darkcover/Documentos/Out/dados/Parte2/matrix.csv')
data.head()

,Unnamed: 0,Listas
0,0,"[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, ..."
1,1,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, ..."
2,2,"[1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, ..."
3,3,"[1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, ..."
4,4,"[1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, ..."


In [3]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

,Listas
0,"[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, ..."
1,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, ..."
2,"[1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, ..."
3,"[1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, ..."
4,"[1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, ..."


In [5]:
data = data.drop(data.index[-1])

In [7]:
data.describe

<bound method NDFrame.describe of                                                 Listas
0    [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, ...
1    [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...
2    [1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, ...
3    [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...
4    [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, ...
..                                                 ...
410  [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, ...
411  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, ...
412  [1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, ...
413  [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, ...
414  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...

[415 rows x 1 columns]>

In [ ]:
# Supondo que seus dados estão em um DataFrame chamado 'df'
# Convertendo a coluna de listas para um array numpy
data = data['Listas'].apply(lambda x: np.array(eval(x))).values
data = np.array([list(item) for item in data])


In [18]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


# Definir a quantidade de entradas usadas para predição
k = 160

# Separar os dados em entradas (X) e saídas (y)
X = data[:, :k]
y = data[:, k:]

# Reformatar X para [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Dividir os dados em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir o modelo LSTM com diferentes arquiteturas
model = Sequential()
model.add(LSTM(640, activation='relu', return_sequences=True, input_shape=(k, 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(y.shape[1], activation='sigmoid'))  # Sigmoid para saída binária
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Adicionar callbacks para early stopping e salvar o melhor modelo
#ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks, verbose=1)

# Fazer predições no conjunto de validação
predictions = (model.predict(X_val) > 0.5).astype(int)  # Converter probabilidades em 0s e 1s

# Avaliar a precisão
accuracy = np.mean(predictions == y_val)
print(f"Validation Accuracy: {accuracy}")


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.0000e+00 - loss: 0.8514 - val_accuracy: 0.0000e+00 - val_loss: 0.6861
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.0000e+00 - loss: 0.6874 - val_accuracy: 0.0000e+00 - val_loss: 0.6878
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.0000e+00 - loss: 0.6873 - val_accuracy: 0.0000e+00 - val_loss: 0.6855
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.0000e+00 - loss: 18.1500 - val_accuracy: 0.0000e+00 - val_loss: 0.6812
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.0000e+00 - loss: 0.6821 - val_accuracy: 0.0000e+00 - val_loss: 0.6825
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.0000e+00 - loss: 0.6820 - val_accuracy: 0.0000e+00 - val_loss: 0.6808
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.0000e+00 - loss: 0.6802 - val_accuracy: 0.0000e+00 - val_loss: 0.6776
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy:

In [1]:
import itertools
import numpy as np

# Definindo o tamanho da sequência e o valor da média desejada
n = 80
target_mean = 0.67
error_margin = 0.05

# Gerando todas as combinações possíveis de sequências binárias
sequences = list(itertools.product([0, 1], repeat=n))

# Lista para armazenar as sequências que atendem ao critério
valid_sequences = []

# Função para calcular a média de uma sequência
def calculate_mean(sequence):
    return np.mean(sequence)

# Iterando sobre todas as sequências
for seq in sequences:
    mean = calculate_mean(seq)
    if abs(mean - target_mean) <= error_margin:
        valid_sequences.append(seq)

# Exemplo de impressão das primeiras 10 sequências válidas
for seq in valid_sequences[:10]:
    print(seq)


: 

In [21]:
X

array([[[1],
        [1],
        [1],
        ...,
        [0],
        [1],
        [0]],

       [[1],
        [1],
        [0],
        ...,
        [0],
        [1],
        [1]],

       [[1],
        [1],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[1],
        [0],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [0],
        [1],
        ...,
        [0],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [0],
        [1],
        [1]]])